In [1]:
from google.cloud import bigquery
import pandas as pd
import os
import time

os.environ['GOOGLE_APPLICATION+CREDENTIALS'] = '/Users/nrabbani/.config/gcloud/application_default_credentials.json'
os.environ['GCLOUD_PROJECT'] = 'mining-clinical-decisions'
%load_ext google.cloud.bigquery

client=bigquery.Client()

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For mor

In [2]:
# Excluding accomodation ECMO and CVVH
query = '''
SELECT * FROM njr_labutil.lradt
    WHERE
        (pat_class = 'Inpatient' OR pat_class = 'Surgery Admit')
        AND pat_service NOT IN (
            'Obstetrics', 'Maternal-Fetal Medicine',
            'Reproductive Endocrinology and Infertility',
            'Gynecology & Obstetrics', 'Gynecology', 'OB Anesthesia')
        AND (pat_lv_of_care = 'Critical Care'
            OR (pat_lv_of_care='' AND accomodation IN (
                'CV Critical Care Level 1', 'CV Critical Care Level 2', 'CV Critical Care Level 3', 
                'Critical Care Level 1', 'Critical Care Level 2', 'Critical Care Level 3', 
                'NICU Level 3 (174)', 'NICU Level 4',
                'Neonatal Level 1 (172)', 'Neonatal Level 2 (173)', 'Neonatal Level 3 (173)'
                )
            )
        )
        AND (accomodation NOT IN ('CVVH', 'ECMO')
        )
'''

In [3]:
query_job = client.query(query)
df = query_job.to_dataframe()

In [4]:
tic = time.time()
lradt = query_job.to_dataframe()
print('Query run time: ', time.time() - tic)

Query run:  1635350925.3993752
Query run time:  1213.1376338005066


In [5]:
lradt.loc[~lradt.ord_num_value.isnull()].to_pickle('data/lradt_icu_20211027.pkl')